In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.special import gammainc

In [2]:
def sigma(sig_ln, F0, alpha):
    ''' Compute sigma'''
    return sig_ln * F0**(1-alpha)

def X(F, alpha, sig):
    ''' X transform variable '''
    return F**(2 * (1-alpha)) / (sig**2 * (1-alpha)**2)

def F_Xinv(X, alpha, sig):
    ''' F from X'''
    return (X * (sig**2 * (1-alpha)**2)) ** (1/(2*(1-alpha)))

def delta(alpha):
    ''' delta from alpha'''
    return (1 - 2*alpha) / (1 - alpha)

def em_CEV(T, dt, F0, alpha, sig):
    ''' Euler scheme simulation '''
    
    # set up Brownian motion and t
    t = np.arange(dt, T + dt, dt)
    N = len(t)
    dW=np.sqrt(dt)*np.random.randn(N)
    
    # init variables
    X0 = X(F0, alpha, sig)
    d = delta(alpha)
    X_emC, X_em = X0, []
    hit_zero = False
    
    # For all timesteps
    for j in range(N):
        # euler (X + XdX)=(X + X * (d*dt + 2X^(1/2)dW))
        dX = d*dt + 2*np.sqrt(X_emC)*dW[j]
        X_emC += dX
        
        # if <= 0, presumed it hit boundary of 0
        if X_emC <= 0:
            # fill with 0's since it is absorbed
            X_em += [0 for i in range(N - len(X_em))]
            hit_zero = True
            break
        X_em.append(X_emC)
    
    return X_em, hit_zero



def mil_CEV(T, dt, F0, alpha, sig):
    ''' Milstein scheme simulation '''
    
    # set up Brownian motion and t
    t = np.arange(dt, T + dt, dt)
    N = len(t)
    dW=np.sqrt(dt)*np.random.randn(N)
    
    # init variables
    X0 = X(F0, alpha, sig)
    d = delta(alpha)
    X_milC, X_mil = X0, []
    hit_zero = False
    
    # For all timesteps
    for j in range(N):
        # Milstein 
        dX = d*dt + 2*np.sqrt(X_milC)*dW[j] + (dW[j]**2 - dt)
        X_milC += dX
        
        # if <= 0, presumed it hit boundary of 0
        if X_milC <= 0:
            # fill with 0's since it is absorbed
            X_mil += [0 for i in range(N - len(X_mil))]
            hit_zero = True
            break
        X_mil.append(X_milC)
    
    return X_mil, hit_zero
    
    

In [3]:
def analytical_C(F,K,T, alpha, sig):
    K_tilde = X(K,alpha, sig)
    X0 = X(F,alpha, sig)
    C = F * (1 - stats.ncx2.cdf(x = K_tilde/T, df = 4 - delta(alpha), nc = X0/T)) - K * stats.ncx2.cdf(x = X0/T, df = 2 - delta(alpha),nc = K_tilde/T)
    
    return C

In [5]:
from scipy.optimize import root
import tqdm
# Parameters
F = 100
alpha = 0
sig_ln = 0.5
T = 4
K = 100
# transform sig to correct dimension
sig = sigma(sig_ln, F, alpha)

# Umax
v = delta(alpha)/2 - 1
X0 = X(F, alpha, sig)
Umax = gammainc(-v, X0/(2*T))

# U
s = stats.qmc.Sobol(1)
M = 20
N = 2**M
U = s.random_base2(M).flatten()


absorbed = U > Umax
print(f"Sim Absorbed: {np.count_nonzero(absorbed)/N}")
print(f"Ana Absorbed: {1 - gammainc(-v, X0/(2*T))}")


XoverT = np.zeros(N)
not_absorbed = ~absorbed
print(X0/T)
f = lambda x, u: stats.ncx2.cdf(X0/T, df = 2 - delta(alpha), nc = x/T) - (u - Umax)
for idx, u in tqdm.tqdm(enumerate(U)):
    if u > Umax:
        continue
    else:
        
        XoverT[idx] =0
        print(root(f, F, args=(u)))
        exit()

X_T = XoverT * T
# compute inverse to get F_T
F_T = F_Xinv(X_T, alpha, sig)
# compute C by subtracting K and taking max. Anywhere less than K is 0, otherwise F - K
C = np.where(F_T<K, 0, F_T-K
             )
P = C - (F_T - K)


print(C.mean())
print(analytical_C(F,K,T,alpha,sig))

Sim Absorbed: 0.3173103332519531
Ana Absorbed: 0.31731050786291415
1.0


0it [00:00, ?it/s]

52it [00:00, 515.80it/s]

 message: The iteration is not making good progress, as measured by the 
            improvement from the last ten iterations.
 success: False
  status: 5
     fun: [ 3.720e-01]
       x: [ 1.010e+04]
    nfev: 13
    fjac: [[ 1.000e+00]]
       r: [-0.000e+00]
     qtf: [ 3.720e-01]
 message: The iteration is not making good progress, as measured by the 
            improvement from the last ten iterations.
 success: False
  status: 5
     fun: [ 1.045e-01]
       x: [ 1.010e+04]
    nfev: 13
    fjac: [[ 1.000e+00]]
       r: [-0.000e+00]
     qtf: [ 1.045e-01]
 message: The iteration is not making good progress, as measured by the 
            improvement from the last ten iterations.
 success: False
  status: 5
     fun: [ 4.843e-01]
       x: [ 1.010e+04]
    nfev: 13
    fjac: [[ 1.000e+00]]
       r: [-0.000e+00]
     qtf: [ 4.843e-01]
 message: The iteration is not making good progress, as measured by the 
            improvement from the last ten iterations.
 success: False
  

128it [00:00, 651.53it/s]

...

 message: The iteration is not making good progress, as measured by the 
            improvement from the last ten iterations.
 success: False
  status: 5
     fun: [ 1.251e-01]
       x: [ 1.010e+04]
    nfev: 13
    fjac: [[ 1.000e+00]]
       r: [-0.000e+00]
     qtf: [ 1.251e-01]
 message: The iteration is not making good progress, as measured by the 
            improvement from the last ten iterations.
 success: False
  status: 5
     fun: [ 3.497e-01]
       x: [ 1.010e+04]
    nfev: 13
    fjac: [[ 1.000e+00]]
       r: [-0.000e+00]
     qtf: [ 3.497e-01]
 message: The iteration is not making good progress, as measured by the 
            improvement from the last ten iterations.
 success: False
  status: 5
     fun: [ 3.345e-01]
       x: [ 1.010e+04]
    nfev: 13
    fjac: [[ 1.000e+00]]
       r: [-0.000e+00]
     qtf: [ 3.345e-01]
 message: The iteration is not making good progress, as measured by the 
            improvement from the last ten iterations.
 success: False
  

7099it [00:10, 651.28it/s]

 message: The iteration is not making good progress, as measured by the 
            improvement from the last ten iterations.
 success: False
  status: 5
     fun: [ 6.190e-02]
       x: [ 1.010e+04]
    nfev: 14
    fjac: [[ 1.000e+00]]
       r: [-0.000e+00]
     qtf: [ 6.190e-02]
 message: The iteration is not making good progress, as measured by the 
            improvement from the last ten iterations.
 success: False
  status: 5
     fun: [ 4.633e-01]
       x: [ 1.010e+04]
    nfev: 13
    fjac: [[ 1.000e+00]]
       r: [-0.000e+00]
     qtf: [ 4.633e-01]
 message: The iteration is not making good progress, as measured by the 
            improvement from the last ten iterations.
 success: False
  status: 5
     fun: [ 5.697e-01]
       x: [ 1.010e+04]
    nfev: 13
    fjac: [[ 1.000e+00]]
       r: [-0.000e+00]
     qtf: [ 5.697e-01]
 message: The iteration is not making good progress, as measured by the 
            improvement from the last ten iterations.
 success: False
  

KeyboardInterrupt: 

: 